In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
zip_path = "/content/drive/MyDrive/ML/data.zip"
unzip_path = "/content/data"
if not os.path.exists(unzip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets, transforms, models
from torchvision.models import ResNet50_Weights
from tqdm import tqdm

In [2]:
# set hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 20

In [ ]:
# data preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(44, padding=4),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=15, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# prepare dataset
# (train)
train_data = datasets.ImageFolder(root=f"{unzip_path}/data/Images/train", transform=transform)
print(f"Total training data: {train_data.__len__()}")

Total training data: 28709


In [ ]:
# split data into train & validate
# train_size = int(0.8 * len(train_data))
# val_size = len(train_data) - train_size
# train_data, val_data = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [3]:
# load pretrain resenet model
weights = ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)
# model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7) # Replace final layer to classify 7 emotions

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]


In [4]:
# move model to GPU (or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
print(f"Using device: {device}")

Using device: cuda


### Train

In [5]:
# loss function & optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(
    model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4
)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of model parameters: {total_params}")

Total number of model parameters: 23522375


In [ ]:
# train loop!
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch + 1}/{num_epochs}")
        for i, (images, labels) in enumerate(tepoch):
          images, labels = images.to(device), labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * images.size(0)
          _, predicted = torch.max(outputs, 1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)

          if i % 10 == 0:  # 每 10 個 batch 更新一次
            tepoch.set_postfix(loss=loss.item(), accuracy=correct/total)
    # Update scheduler
    scheduler.step()
    # print this epoch result
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    # Validation Loop
    # model.eval()
    # val_running_loss = 0.0
    # val_correct = 0
    # val_total = 0
    # with torch.no_grad():
    #     with tqdm(val_loader, unit="batch") as vepoch:
    #         vepoch.set_description(f"Validation {epoch + 1}/{num_epochs}")
    #         for images, labels in vepoch:
    #             images, labels = images.to(device), labels.to(device)

    #             outputs = model(images)
    #             loss = criterion(outputs, labels)

    #             val_running_loss += loss.item() * images.size(0)
    #             _, predicted = torch.max(outputs, 1)
    #             val_correct += (predicted == labels).sum().item()
    #             val_total += labels.size(0)

    #             vepoch.set_postfix(loss=loss.item(), accuracy=val_correct/val_total)

    # val_epoch_loss = val_running_loss / val_total
    # val_epoch_acc = val_correct / val_total
    # print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.4f}")
    # # Save best model
    # if val_epoch_acc > best_val_acc:
    #     best_val_acc = val_epoch_acc
    #     torch.save(model.state_dict(), "resnet_with_scheduler.pth")
    #     print("Best model saved!")

Epoch 1/20: 100%|██████████| 898/898 [00:53<00:00, 16.66batch/s, accuracy=0.764, loss=0.506]


Epoch 1/20, Loss: 0.6693, Accuracy: 0.7637


Epoch 2/20: 100%|██████████| 898/898 [00:53<00:00, 16.90batch/s, accuracy=0.763, loss=0.503]


Epoch 2/20, Loss: 0.6686, Accuracy: 0.7634


Epoch 3/20: 100%|██████████| 898/898 [00:56<00:00, 15.79batch/s, accuracy=0.77, loss=1]


...

Epoch 18/20, Loss: 0.4268, Accuracy: 0.8476


Epoch 19/20: 100%|██████████| 898/898 [00:54<00:00, 16.48batch/s, accuracy=0.861, loss=0.468]


Epoch 19/20, Loss: 0.3919, Accuracy: 0.8604


Epoch 20/20: 100%|██████████| 898/898 [00:53<00:00, 16.80batch/s, accuracy=0.863, loss=0.413]

Epoch 20/20, Loss: 0.3811, Accuracy: 0.8628


In [ ]:
# save trained model
torch.save(model.state_dict(), "/content/drive/MyDrive/ML/resnet_no_validate.pth")
print("Training complete. Model saved as 'resnet_no_validate.pth'.")

Training complete. Model saved as 'resnet_no_validate.pth'.


###Evaluate

In [ ]:
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # 將灰階圖片轉為 3 通道
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # RGB 通道的 Normalize
])

In [ ]:
from PIL import Image
import glob

class TestingDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.names = []

        self.images = sorted(glob.glob(f"{self.img_dir}/*"))
        self.names = [os.path.basename(image)[:-4] for image in self.images]

    def __len__(self):
        return len(self.images)

    def __getnames__(self):
        return self.names

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("L")
        image = self.transform(image)
        return image

In [ ]:
test_data = TestingDataset(f"{unzip_path}/data/Images/test", test_transform)
print(f"Total testing data: {test_data.__len__()}")
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Total testing data: 3589


In [ ]:
# Load the trained model and evaluate
model.load_state_dict(torch.load("/content/drive/MyDrive/ML/resnet_no_validate.pth"))

<ipython-input-31-0f5be5a424c0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/ML/resnet_no_validate.pth"))


<All keys matched successfully>

In [ ]:
import pandas as pd

model.eval()
predictions = []
with torch.no_grad():
  for i, images in enumerate(tqdm(test_loader)):
    images = images.to(device)
    logits = model(images)
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

# 預設的 index_mapping
index_mapping = {"Angry": 0, "Disgust": 1, "Fear": 2, "Happy": 3, "Neutral": 4, "Sad": 5, "Surprise": 6}

# 將預測索引轉換為對應類別名稱
class_dic = {v: k for k, v in index_mapping.items()}
predictions = [class_dic[pred] for pred in predictions]  # 索引 -> 類別名稱

# 模擬測試資料的檔案名稱
filenames = test_data.__getnames__()

# 將類別名稱轉為指定的數字標籤
labels = [index_mapping[pred] for pred in predictions]

# 建立 DataFrame 並輸出到 CSV
submission = pd.DataFrame({"filename": filenames, "label": labels})
submission.to_csv("/content/drive/MyDrive/ML/resnet_no_validate.csv", index=False)

100%|██████████| 113/113 [00:05<00:00, 21.87it/s]
